In [1]:
pip install telethon pandas requests langdetect


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install nest_asyncio

Note: you may need to restart the kernel to use updated packages.


In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except ModuleNotFoundError:
    pass


In [6]:
import os, json, csv, time, re
import asyncio, requests, pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from urllib.parse import urlparse
from langdetect import detect, DetectorFactory
DetectorFactory.seed=0
load_dotenv()


False

In [7]:
from telethon import TelegramClient, errors
from telethon.tl.functions.channels import JoinChannelRequest


In [8]:
from telethon.sync import TelegramClient
from getpass import getpass

In [9]:
API_ID  = 25745274                       
API_HASH = '9652fe84b03043cdbbd0e1c36f25d5a9'
PHONE    = '+8613585795327'


#back up account
# API_ID  = 24527187                       
# API_HASH = '8a012769064c0218d779c9e23e45fdb0'
# PHONE    = '+18573343025'



import os
os.environ["API_ID"]   = str(API_ID)      # must be strings
os.environ["API_HASH"] = API_HASH
os.environ["PHONE"]    = PHONE


In [10]:
# expects API_ID, API_HASH, and PHONE in the environment
API_ID = os.getenv("API_ID")
API_HASH = os.getenv("API_HASH")
PHONE = os.getenv("PHONE")
if not all([API_ID, API_HASH, PHONE]):
    raise RuntimeError("API_ID, API_HASH and PHONE environment variables must be set")


In [11]:
import re

# Matches @handles that are at least 5 characters, digits or '_'
USERNAME_RE  = re.compile(r'@([A-Za-z0-9_]{5,})')

# You already used this for text-link harvesting
TME_LINK_RE  = re.compile(r'(?:t\.me/|@)([A-Za-z0-9_]{5,})')
TMELINK_RE   = TME_LINK_RE


In [12]:
import re, asyncio
from telethon.errors import FloodWaitError

# ---------- simple username validation ----------
VALID_UNAME = re.compile(r'^@[A-Za-z0-9_]{5,32}$')

def looks_like_username(s: str) -> bool:
    return bool(VALID_UNAME.match(s.strip()))

# ---------- global cache ----------
_entity_cache: dict[str, "telethon.tl.types.User|Channel|Chat"] = {}

# ---------- safe resolver ----------
async def safe_get_entity(client, handle: str):
    """
    Resolve @username → Telethon entity, with cache and Flood-Wait handling.
    Returns None if the handle is malformed, deleted/private, or would trigger
    a Flood-Wait > 10 min.
    """
    handle = handle.strip()

    # reject obviously bad strings before touching the API
    if not looks_like_username(handle):
        return None

    # cached hit
    if handle in _entity_cache:
        return _entity_cache[handle]

    try:
        # **direct call to Telegram once**
        ent = await client.get_entity(handle)
        _entity_cache[handle] = ent
        return ent

    except FloodWaitError as e:
        if e.seconds > 600:                    # long wait → skip handle
            print("Skip", handle, "— FloodWait", e.seconds, "s")
            return None
        print("Sleeping", e.seconds, "s for", handle)
        await asyncio.sleep(e.seconds + 2)
        # retry once after short wait
        return await safe_get_entity(client, handle)

    except Exception:
        # invalid / deleted / private
        return None


In [13]:
PROGRESS_JSON = os.getenv('PROGRESS_JSON', '/content/drive/MyDrive/telegram/progress.json')
RAW_CSV = os.getenv('RAW_CSV', '/content/drive/MyDrive/telegram/messages2.csv')
CLEAN_CSV = os.getenv('CLEAN_CSV', '/content/drive/MyDrive/telegram/cleaned.csv')
SESSION_NAME = os.getenv('SESSION_NAME', 'tg_session')

In [14]:
BFS_HOPS              = 4      # how many BFS layers (depth)
BFS_CHANNEL_MSG_LIMIT = 250    # deeper scan for channels
BFS_GROUP_MSG_LIMIT   = 20     # smaller scan for groups to avoid excessive overhead
BFS_MIN_SUBS          = 0      # no minimum subscriber threshold
BFS_LANG_SAMPLE       = 3000   # messages sampled to check language
BFS_EN_RATIO          = 0.2    # 20% must be English
BFS_FWD_MIN           = 0      # min forwards to count a channel
BFS_INCLUDE_MENTIONS  = True   # parse @username / t.me links
BFS_TOPIC_KEYWORDS    = []     # keywords to filter topics (not used if empty)
BFS_TOPIC_RATIO       = 0      # ratio threshold for keyword matches
BFS_TOPIC_SAMPLE      = 20     # how many messages sampled for topic ratio
BFS_MAX_DORMANT_DAYS  = 1825   # ignore channels dormant > X days


# Global set to track which user IDs we've already scanned (for participant bounce)
scanned_user_ids = set()

In [15]:
# ===== Throttling helper ==================================================
import asyncio, time, random
from telethon import errors as _tl_errors

RATE_LIMIT = 1.2      # ≥ 1 s between any two RPCs – raise if you still flood
JITTER     = 0.3      # add ± up to 0.3 s so calls look “human”
_last_call = 0.0

async def safe_api_call(coro, *args, **kwargs):
    """
    Call ANY Telethon coroutine (or the `client` object itself) while
    1⃣  spacing requests, and 2⃣  automatically respecting FloodWait.
    """
    global _last_call
    gap = RATE_LIMIT + random.uniform(-JITTER, JITTER)
    now = time.time()
    if now - _last_call < gap:
        await asyncio.sleep(gap - (now - _last_call))
    _last_call = time.time()

    try:
        return await coro(*args, **kwargs)
    except _tl_errors.FloodWaitError as e:
        wait = e.seconds + 2               # +2 s safety pad
        print(f"⚠️  FloodWait {e.seconds}s → sleeping {wait}s …")
        await asyncio.sleep(wait)
        _last_call = time.time()
        return await coro(*args, **kwargs)  # one retry is enough


In [16]:
# --- numeric-ID bulk-resolution globals ---------------------------
pending_ids: dict[int, tuple[int, int]] = {}   # cid → (access_hash, parent_hop)
RESOLVE_INTERVAL = 300      # resolve after this many queued ids
MAX_PER_CALL     = 100      # ≤100 ids per GetChannelsRequest  (Telegram soft cap)


In [17]:
# Throttle settings --------------------------------------------------------
import random, asyncio
BFS_PAUSE_SEC   = 0.8        # base delay after each 100-msg pull
BFS_BATCH_SIZE  = 100        # pause every 100 messages
# -------------------------------------------------------------------------

async def _batched_iter_messages(entity, *, limit=None):
    """
    Iterator that keeps us well below Telegram’s
    ≈10 GetHistory-calls per 30 s soft limit.
    """
    i = 0
    async for msg in client.iter_messages(entity, limit=limit):
        yield msg
        i += 1
        if i % BFS_BATCH_SIZE == 0:
            await asyncio.sleep(BFS_PAUSE_SEC + random.random()*0.4)


In [18]:
channels_df = pd.read_csv('Final_tele.csv')
def _parse_subs(s):
    s = str(s).replace('+', '').strip()
    mult = 1
    if s.lower().endswith('k'):
        mult = 1000
        s = s[:-1]
    elif s.lower().endswith('m'):
        mult = 1000000
        s = s[:-1]
    try:
        return int(float(s) * mult)
    except ValueError:
        return 0
channels_df['subs_num'] = channels_df['Subscribers'].apply(_parse_subs)
seed_usernames = channels_df['Username'].tolist()
FOLLOWERS_DICT = channels_df.set_index('Username')['subs_num'].to_dict()

In [19]:
CHANNELS = seed_usernames

In [20]:
client=TelegramClient(SESSION_NAME,API_ID,API_HASH)
def load_progress():
  if os.path.exists(PROGRESS_JSON):
    with open(PROGRESS_JSON,'r')as f:return json.load(f)
  return {}


In [21]:
def save_progress(d):
  os.makedirs(os.path.dirname(PROGRESS_JSON),exist_ok=True)
  with open(PROGRESS_JSON,'w')as f:json.dump(d,f)
  print("Progress saved.")


In [22]:

def append_header_if_needed():
    ex = os.path.exists(RAW_CSV)
    os.makedirs(os.path.dirname(RAW_CSV), exist_ok=True)
    if not ex:
        with open(RAW_CSV, 'w', newline='', encoding='utf-8') as f:
            # Note: 'channel_num' added to the header
            csv.writer(f).writerow([
                'channel', 
                'msg_id', 
                'date', 
                'text', 
                'fwd_from', 
                'fwd_id', 
                'bfs_cycle',
                'channel_num'
            ])


In [23]:
async def join_if_needed(ent):
  if getattr(ent,'megagroup',False):
    try:await client(JoinChannelRequest(ent))
    except:pass


In [24]:
# --------------------------------------------------------------------
#  scrape_channel – flood-wait-aware, entity-optional
# --------------------------------------------------------------------
async def scrape_channel(
        name: str,
        last_id: int,
        bfs_depth: int,
        channel_num: int,
        *,
        entity=None,
        msg_limit: int | None = None
    ) -> int:

    if entity is None:                       # resolve if caller didn’t pass it
        entity = await safe_get_entity(client, name)
        if entity is None:
            print("Skip", name, "— cannot resolve")
            return last_id
        await asyncio.sleep(1.1)

    await join_if_needed(entity)

    max_id  = last_id
    fetched = 0

    try:
        async for msg in _batched_iter_messages(entity, limit=msg_limit):
            if msg.id <= last_id:
                continue
            if msg_limit and fetched >= msg_limit:
                break
            fetched += 1
            max_id = max(max_id, msg.id)

            # ---------- CSV row ----------
            fw_title, fw_id = None, None
            if msg.forward:
                if msg.forward.chat:
                    fw_title = msg.forward.chat.title or msg.forward.chat.username
                    fw_id    = getattr(msg.forward.chat, 'id', None)
                elif msg.forward.sender:
                    fw_title = (msg.forward.sender.first_name or
                                msg.forward.sender.username or 'Private')
                    fw_id    = getattr(msg.forward.sender, 'id', None)

            with open(RAW_CSV, "a", newline='', encoding="utf-8") as f:
                csv.writer(f).writerow([
                    name, msg.id, msg.date.isoformat(), msg.text or '',
                    fw_title, fw_id, bfs_depth, channel_num
                ])

    except FloodWaitError as e:
        print("FloodWait while reading", name, "→ stop (", e.seconds, "s )")

    except Exception as e:
        print("Error reading", name, e.__class__.__name__)

    print(name, f"+{max_id - last_id} msgs; up to", max_id)
    return max_id


In [25]:
def run_scrape():
  asyncio.run(scrape_once())
  print("Scraping done.")


In [26]:
def deduplicate(messages):
  out=[];seen=set()
  for m in messages:
    k=(m.get('fwd_id'),m.get('text'))
    if m.get('fwd_id'):
      if k in seen:continue
      seen.add(k)
    out.append(m)
  return out


In [27]:
def unshorten_url(u,t=5):
  try:return requests.head(u,allow_redirects=True,timeout=t).url
  except:return u


In [28]:
def expand_and_extract(m):
  txt=m['text'];urls=re.findall(r'https?://\S+',txt)
  dom=set()
  for u in urls:
    fin=unshorten_url(u);d=urlparse(fin).netloc
    dom.add(d)
    txt=txt.replace(u,fin)
  m['text']=txt;m['domains']=list(dom);return m


In [29]:
def is_english(txt):
  if not txt.strip():return False
  try:return(detect(txt)=='en')
  except:return False


In [30]:
def anonymize(m,user_map):
  if m.get('fwd_id')and m['fwd_from']and not str(m['fwd_from']).startswith(('Channel','@')):
    if m['fwd_id'] not in user_map:user_map[m['fwd_id']]=f"User{len(user_map)+1}"
    m['fwd_from']=user_map[m['fwd_id']];m['fwd_id']=None
  m['text']=re.sub(r'@[\w\d_]+','@USER',m['text'])
  return m


In [31]:
def preprocess_data():
  if not os.path.exists(RAW_CSV):print("No raw CSV.");return
  df=pd.read_csv(RAW_CSV,names=['chan','msg_id','date','text','fwd_from','fwd_id'],header=0)
  msgs=df.to_dict('records');msgs=deduplicate(msgs)
  out=[];umap={}
  for m in msgs:
    m=expand_and_extract(m)
    if is_english(m['text']):m=anonymize(m,umap);out.append(m)
  pd.DataFrame(out).to_csv(CLEAN_CSV,index=False)
  print(f"Saved {len(out)} cleaned msgs to {CLEAN_CSV}")


In [32]:
# Load existing progress data and merge new seeds
progress = load_progress()
for username in seed_usernames:
    if username not in progress:
        progress[username] = 0  # mark as not yet scraped


In [33]:
from pathlib import Path
import os

# Where should we store run-time artefacts?
WORKDIR = Path(os.getenv("TG_WORKDIR", Path.cwd() / "telegram_data"))
WORKDIR.mkdir(parents=True, exist_ok=True)   # makes ./telegram_data if absent

PROGRESS_JSON = WORKDIR / "progress.json"
RAW_CSV       = WORKDIR / "messages.csv"
CLEAN_CSV     = WORKDIR / "cleaned.csv"

In [34]:
# Persist updated progress
save_progress(progress)

Progress saved.


In [35]:
from langdetect import detect
async def _en_ratio(ent, sample=BFS_LANG_SAMPLE):
    en = 0
    async for m in client.iter_messages(ent, limit=sample):
        t = (m.message or '').strip()
        if t:
            try:  en += detect(t)=='en'
            except: pass
    return 0 if not sample else en/sample


In [36]:
# Example : _forward_sources  (cell 33) ------------------------------
async def _forward_sources(ent, limit, parent_hop):
    """
    Scan recent messages for official forwards.
    Returns a set of discovered numeric channel_ids and records them in `pending_ids`.
    """
    global pending_ids
    counts = {}

    async for m in _batched_iter_messages(ent, limit=limit):
        pc = m.forward.chat or m.forward.original_fwd if m.forward else None
        if isinstance(pc, PeerChannel):
            cid, ah = pc.channel_id, pc.access_hash
            if cid and ah:
                pending_ids.setdefault(cid, (ah, parent_hop))
            counts[cid] = counts.get(cid, 0) + 1

    return {cid for cid, c in counts.items() if c >= BFS_FWD_MIN}


In [37]:
# --- Bulk numeric-id → @username resolver ---------------------------------
from telethon.tl.functions.channels import GetChannelsRequest
from telethon.tl.types import InputChannel

async def bulk_resolve_channels(client) -> list[tuple[str, int]]:
    """
    Turn queued numeric ids in `pending_ids` into canonical '@handles'
    in batches of ≤MAX_PER_CALL, obeying safe_api_call().
    Returns list of tuples: (username, depth).
    """
    global pending_ids
    if not pending_ids:
        return []

    items = list(pending_ids.items())
    pending_ids.clear()

    resolved = []
    for i in range(0, len(items), MAX_PER_CALL):
        batch  = items[i:i+MAX_PER_CALL]
        inputs = [InputChannel(cid, ah) for cid, (ah, _) in batch]

        res = await safe_api_call(client, GetChannelsRequest(id=inputs))

        depth_of = {cid: parent for cid, (_, parent) in batch}
        for ch in res.chats:
            if getattr(ch, "username", None):
                uname = '@' + ch.username.lower()
                resolved.append((uname, depth_of.get(ch.id, 0) + 1))

        # courteous pause between big RPCs
        await asyncio.sleep(BFS_PAUSE_SEC + random.random()*0.4)

    return resolved


In [38]:
import re

MENTION_RE = re.compile(r'@([\w\d_]{3,32})|t.me/([\w\d_]{3,32})', re.IGNORECASE)

# Additional patterns for textual forward references
FORWARDED_FROM_RE = re.compile(
    r'(?:\[?\s*forwarded\s+from\s+([\w\d_.-]+)\]?)',
    re.IGNORECASE
)

# e.g. "Forwarded from Disclose.tv" or "[Forwarded from MyChannel]"
# This is naive: we assume channel-like text is one token (no spaces).
# If the channel doesn't actually exist, it won't pass _is_valid() anyway.
EXT_REF_RE = re.compile(
    r'(?:credit|source|via|thanks\s+to|shared\s+from|copied\s+from)\s*[:-]?\s*@?([\w\d_]{3,32})',
    re.IGNORECASE
)

# --- Cell 34  (REPLACE) ---------------------------------------------
async def _mention_sources(ent, limit):
    """
    Look for @mentions, t.me links, 'Forwarded from …', 'Source: …', etc.
    Returns a set of @usernames (with leading '@').
    """
    src = set()
    if not BFS_INCLUDE_MENTIONS:
        return src

    async for m in _batched_iter_messages(ent, limit=limit):
        text = m.message or ""

        # classic @mention or t.me/…
        for a, b in MENTION_RE.findall(text):
            name = a or b
            if name:
                src.add("@" + name.lstrip("@"))

        # "Forwarded from Disclose.tv"
        for fwd in FORWARDED_FROM_RE.findall(text):
            candidate = fwd.strip()
            if " " not in candidate:
                src.add("@" + candidate.replace(".", "").lstrip("@"))

        # "Source: @ChannelName"
        for e2 in EXT_REF_RE.findall(text):
            src.add("@" + e2.lstrip("@"))

    return src


In [39]:
import re, asyncio
TME_LINK_RE = re.compile(r"(?:t\.me/|@)([A-Za-z0-9_]{5,})")

async def _link_sources(ent, limit=100):
    """
    Harvest plain-text @handles or t.me links from the last `limit` messages.
    Returns a set of @usernames (with leading '@').
    """
    src = set()
    async for m in _batched_iter_messages(ent, limit=limit):
        text = m.message or ""
        for handle in TME_LINK_RE.findall(text):
            src.add("@" + handle.lstrip("@"))
    return src

In [40]:
async def _to_username(peer):
    """
    Convert any Telethon Peer / id / string into a canonical '@username'.
    Returns None if it cannot be resolved.
    """
    if isinstance(peer, str):
        return '@' + peer.lstrip('@')

    try:
        ent = await safe_api_call(client.get_entity, peer)
        uname = getattr(ent, 'username', None)
        if uname:
            return '@' + uname.lstrip('@')
    except Exception:
        pass
    return None


In [41]:
async def _topic_ratio(ent, keywords, sample):
    """
    Checks what fraction of the last `sample` messages contain any of the keywords.
    If keywords is empty, returns 1.0 to skip.
    """
    if not keywords:
        return 1.0
    pat = re.compile('|'.join(re.escape(k) for k in keywords), re.I)
    hits = 0
    tot = 0
    async for m in client.iter_messages(ent, limit=sample):
        if BFS_PAUSE_SEC:
            await asyncio.sleep(BFS_PAUSE_SEC)
        text = (m.message or '')
        if text.strip():
            tot += 1
            if pat.search(text):
                hits += 1
    return 0 if not tot else hits / tot

In [42]:
from datetime import datetime, timezone


async def _is_valid(ent, *, debug=False):
    """Return (is_ok , reasons_list)."""
    reasons = []

    # 1. subscriber count
    count = getattr(ent, "participants_count", 0) or getattr(ent, "members_count", 0)
    if count < BFS_MIN_SUBS:
        reasons.append(f"subs={count} < {BFS_MIN_SUBS}")

    # 2. language ratio
    en_ratio = await _en_ratio(ent)
    if en_ratio < BFS_EN_RATIO:
        reasons.append(f"en_ratio={en_ratio:.2f} < {BFS_EN_RATIO}")

    # 3. whatever other tests you have …
    # if …: reasons.append("some other test")

    ok = not reasons
    if debug and reasons:
        print("    ✖ REJECT", getattr(ent, "username", "?"), " | ".join(reasons), flush=True)

    return ok, reasons


In [43]:
async def snowball_discovery():
    global CHANNELS
    progress = load_progress()

    # 1) Normalize existing CHANNELS so all have leading '@'
    seeds = []
    for c in CHANNELS:
        c = c.strip()
        if not c.startswith('@'):
            c = '@' + c
        seeds.append(c)

    # 2) Discover BFS
    validated, all_discovered, depth_map = await discover_bfs(seeds, max_hops=BFS_HOPS)

    # 3) Mark validated channels in progress
    for v in validated:
        if v not in progress:
            progress[v] = 0
    save_progress(progress)

    # 4) Merge newly validated channels, again force '@'
    merged = set()
    for ch in (list(CHANNELS) + list(validated)):
        ch = ch.strip()
        if not ch.startswith('@'):
            ch = '@' + ch
        merged.add(ch)

    CHANNELS = list(merged)

    # 5) Summary
    print(
        'discovered:', len(all_discovered),
        '| validated:', len(validated),
        '| total CHANNELS:', len(CHANNELS)
    )

    return depth_map

In [44]:
# --------------------------------------------------------------------
#  scrape_once – rate-safe, flood-wait-aware version
# --------------------------------------------------------------------
from telethon.errors import FloodWaitError

async def scrape_once():
    """Run one full cycle: BFS discovery ➜ per-channel scraping ➜ save progress."""
    await client.start(phone=PHONE)

    # 1 ▸ Discover new channels and get their BFS depth
    depth_map = await snowball_discovery()

    # 2 ▸ Load previous scraping progress
    last = load_progress()

    # 3 ▸ Make sure the CSV header exists
    append_header_if_needed()

    # 4 ▸ Scrape each validated channel
    total = len(CHANNELS)
    for idx, handle in enumerate(CHANNELS, 1):
        print(f"[{idx}/{total}] Scraping channel: {handle}")

        # ---------- safe resolver (cache + flood-wait aware) ----------
        ent = await safe_get_entity(client, handle)
        if ent is None:                      # bad handle or 11h FloodWait
            continue
        await asyncio.sleep(1.1)            # keep resolve rate < 1 req/s
        # ----------------------------------------------------------------

        # depth of this handle in the discovery graph
        bfs_depth = depth_map.get(handle, 0)

        # ---------- download messages with its own flood-wait guard -----
        try:
            last[handle] = await scrape_channel(
                name=handle,
                last_id=last.get(handle, 0),
                bfs_depth=bfs_depth,
                channel_num=idx,
                entity=ent                       # pass the resolved entity
            )

        except FloodWaitError as e:
            if e.seconds > 600:                 # long wait → skip the channel
                print(f"Skip {handle} — FloodWait {e.seconds}s")
                continue
            print(f"Sleeping {e.seconds}s while reading {handle}")
            await asyncio.sleep(e.seconds + 2)
            # one retry (optional)
            last[handle] = await scrape_channel(
                name=handle,
                last_id=last.get(handle, 0),
                bfs_depth=bfs_depth,
                channel_num=idx,
                entity=ent
            )
        # ----------------------------------------------------------------

        if BFS_PAUSE_SEC:
            await asyncio.sleep(BFS_PAUSE_SEC)

    # 5 ▸ Persist progress and disconnect cleanly
    save_progress(last)
    await client.disconnect()
# --------------------------------------------------------------------


In [45]:
def main():
  run_scrape()
  preprocess_data()
  print("Done.")


In [46]:
# =================================================================================
#  Helper regexes, cache, safe_get_entity, and the patched discover_bfs()
# =================================================================================
import re, asyncio
from collections import deque
from telethon.errors import FloodWaitError
from telethon.tl.types import Channel, Chat

# ---------- reusable regexes -----------------------------------------------------
VALID_UNAME  = re.compile(r'^@[A-Za-z0-9_]{5,32}$')             # canonical @handle
TME_LINK_RE  = re.compile(r'(?:t\.me/|@)([A-Za-z0-9_]{5,})')    # link extractor

# ---------- @handle validator --------------------------------------------
VALID_UNAME = re.compile(r'^@[A-Za-z0-9_]{5,32}$')
def looks_like_username(s: str) -> bool:
    return bool(VALID_UNAME.match(s.strip()))

_entity_cache: dict[str, Channel | Chat] = {}

async def safe_get_entity(client, handle: str):
    """
    Cached @username → entity resolver that runs through safe_api_call.
    Returns None on bad format, private/deleted channels, or long waits.
    """
    handle = handle.strip()
    if not looks_like_username(handle):
        return None

    if handle in _entity_cache:
        return _entity_cache[handle]

    try:
        ent = await safe_api_call(client.get_entity, handle)
        _entity_cache[handle] = ent
        return ent
    except Exception:
        return None          # includes any FloodWait already handled upstream


In [47]:
# =================================================================================
#  discover_bfs()  – numeric-ID aware, bulk-resolving version
# =================================================================================
async def discover_bfs(
        seeds,
        *,
        max_hops: int = BFS_HOPS,
        scanned_user_ids: set | None = None,    # for _participants_bounce
    ):
    """
    Breadth-first discovery across Telegram channels / groups.
    Returns (validated_set, all_discovered_set, depth_map).
    """
    print("### ENTER discover_bfs ###", flush=True)

    # ---------- initial frontier --------------------------------------
    visited        = set(seeds)
    queue          = deque((s, 0) for s in seeds)   # (handle, hop)
    validated      = set()
    all_discovered = set(seeds)
    depth_map      = {s: 0 for s in seeds}

    if scanned_user_ids is None:
        scanned_user_ids = set()

    # ---------- BFS loop ----------------------------------------------
    while queue:
        print("--- TOP OF LOOP ---", queue[0][0], flush=True)
        print(
            f"    progress → validated={len(validated):>4} "
            f"visited={len(visited):>4} queue={len(queue):>3}",
            flush=True
        )

        username, hop = queue.popleft()
        if hop >= max_hops:
            continue

        # 1) entity lookup (cached, rate-safe)
        ent = await safe_get_entity(client, username)
        if ent is None:
            continue

        all_discovered.add(username)

        # 2) quick type flags
        is_channel = getattr(ent, "broadcast", False)
        is_group   = getattr(ent, "megagroup", False) or isinstance(ent, Chat)

        # 3) validation
        is_ok, _ = await _is_valid(ent, debug=True)
        if not is_ok:
            continue
        validated.add(username)

        # 4) how many messages to scan
        msg_limit = BFS_GROUP_MSG_LIMIT if is_group else BFS_CHANNEL_MSG_LIMIT

        # 5) neighbour harvest ----------------------------------------------------
        neighbors = set()

        fwd_sources     = await _forward_sources(ent, msg_limit, hop)
        mention_sources = await _mention_sources(ent, msg_limit)
        link_text       = await _link_sources(ent, msg_limit)
        linked          = await _linked_discussion(client, ent)
        pinned_about    = await _links_in_about_and_pinned(client, ent)
        invites         = await _invite_links_in_messages(client, ent, msg_limit)
        inline_entities = await _parse_message_entities(client, ent, msg_limit)
        button_links    = await _parse_button_links(client, ent, msg_limit)

        neighbors |= fwd_sources | mention_sources | link_text | linked
        neighbors |= pinned_about | invites | inline_entities | button_links

        if is_group:
            neighbors |= await _participants_bounce(
                client, ent, scanned_user_ids, limit=msg_limit
            )

        # -- debug ------------------------------------------------------
        print(
            f"[DBG] {username}: {len(neighbors)} raw neighbours "
            f"({len(fwd_sources)} fwd, {len(mention_sources)} mention, "
            f"{len(link_text)} textlink, {len(linked)} linked, "
            f"{len(pinned_about)} about/pinned, {len(invites)} invites, "
            f"{len(inline_entities)} entities, {len(button_links)} buttons)",
            flush=True
        )

        # 6) canonicalise neighbours → '@handle', enqueue unseen ones ----
        neighbors_clean = set()

        # (a) numeric IDs from fwd_sources were already stored in pending_ids;
        #     we'll resolve them in bulk later, so skip resolving here.

        # (b) everything else: try to resolve now (cheap path)
        for nb in neighbors:
            if isinstance(nb, int):           # pure numeric ID → skip
                continue
            ent_nb = await safe_get_entity(client, nb)
            if ent_nb and ent_nb.username:
                neighbors_clean.add("@" + ent_nb.username.lower())

        for neigh in neighbors_clean:
            if neigh not in visited:
                visited.add(neigh)
                depth_map[neigh] = hop + 1
                queue.append((neigh, hop + 1))
                all_discovered.add(neigh)

        # 7) if we have accumulated a lot of numeric IDs, resolve them now
        if len(pending_ids) >= RESOLVE_INTERVAL:
            newly = await bulk_resolve_channels(client)
            for uname, dep in newly:
                if uname not in visited:
                    visited.add(uname)
                    depth_map[uname] = dep
                    queue.append((uname, dep))
                    all_discovered.add(uname)

    # ---------- flush whatever numeric IDs remain ----------------------
    newly = await bulk_resolve_channels(client)
    for uname, dep in newly:
        if uname not in visited:
            visited.add(uname)
            depth_map[uname] = dep
            # may exceed max_hops; they'll be crawled next run
            all_discovered.add(uname)

    # ---------- summary banner ----------------------------------------
    print(
        f"### DONE: validated={len(validated)}  "
        f"all_discovered={len(all_discovered)}  "
        f"max_depth={max(depth_map.values())}",
        flush=True
    )
    return validated, all_discovered, depth_map
# =================================================================================


In [48]:
import re
import asyncio

from telethon.tl.functions.channels import GetFullChannelRequest
from telethon.tl.functions.contacts import ResolveUsernameRequest
from telethon.tl.types import (
    MessageEntityTextUrl,
    MessageEntityMention,
    MessageEntityMentionName,
    PeerChannel
)
from datetime import datetime  # if needed for dormancy checks

In [49]:
async def _get_full_channel(client, ent):
    """
    A small helper that wraps GetFullChannelRequest.
    Returns full_chat data or None if something fails.
    """
    try:
        return await client(GetFullChannelRequest(ent))
    except:
        return None

In [50]:
async def _linked_discussion(client, ent):
    """
    Check if this channel has a linked discussion group (linked_chat_id).
    Returns a set with the linked group's username if found, else empty set.
    """
    found = set()
    full = await _get_full_channel(client, ent)
    if full and full.full_chat.linked_chat_id:
        try:
            linked_ent = await client.get_entity(full.full_chat.linked_chat_id)
            if getattr(linked_ent, 'username', None):
                found.add(linked_ent.username)
        except:
            pass
    return found

In [51]:
async def _links_in_about(client, ent):
    """
    Extract any @usernames / t.me links from a channel's 'about' text.
    Returns a set of discovered usernames.
    """
    found = set()
    full = await _get_full_channel(client, ent)
    if not full:
        return found

    about = (full.full_chat.about or "").strip()
    # match @username
    for m in USERNAME_RE.findall(about):
        found.add(m)
    # match t.me/... patterns
    for m in TMELINK_RE.findall(about):
        found.add(m.lstrip("+/"))  # remove leading '+' or 'joinchat/'

    return found

In [52]:
async def _links_in_pinned_message(client, ent):
    """
    If the channel has a pinned message, scan it for @usernames or t.me links.
    Returns a set of discovered usernames.
    """
    found = set()
    full = await _get_full_channel(client, ent)
    if not full:
        return found

    if full.full_chat.pinned_msg_id:
        try:
            pinned_msg = await client.get_messages(ent, ids=full.full_chat.pinned_msg_id)
            pinned_text = (pinned_msg.message or "").strip()
            # match @username
            for m in USERNAME_RE.findall(pinned_text):
                found.add(m)
            # match t.me/...
            for m in TMELINK_RE.findall(pinned_text):
                found.add(m.lstrip("+/"))
        except:
            pass

    return found

In [53]:
async def _links_in_about_and_pinned(client, ent):
    """
    Aggregates results from _links_in_about + _links_in_pinned_message.
    Returns a combined set of discovered usernames.
    """
    found = set()
    about_links = await _links_in_about(client, ent)
    pinned_links = await _links_in_pinned_message(client, ent)
    found |= about_links
    found |= pinned_links
    return found

In [54]:
def _extract_invite_slugs(text):
    """
    Finds all t.me/... patterns in a piece of text, ignoring leading +/ in the slug.
    Returns a list of invite slugs (strings).
    """
    slugs = []
    for match in TMELINK_RE.findall(text):
        slug = match.lstrip("+/")
        if slug:
            slugs.append(slug)
    return slugs

In [55]:
async def _resolve_invite_slugs(client, slugs):
    """
    Given a list of invite slugs, attempt to ResolveUsernameRequest for each.
    Return a set of discovered channel usernames.
    """
    found = set()
    for slug in slugs:
        try:
            r = await client(ResolveUsernameRequest(slug))
            for ch in r.chats:
                if getattr(ch, 'username', None):
                    found.add(ch.username)
        except:
            pass
    return found

In [56]:
async def _invite_links_in_messages(client, ent, limit=100000):
    """
    Extract t.me/… slugs, resolve them in bulk, return discovered usernames.
    Works over the last `limit` messages.
    """
    found = set()

    async for m in _batched_iter_messages(ent, limit=limit):
        text = m.message or ""
        slugs = _extract_invite_slugs(text)
        if not slugs:
            continue
        resolved = await _resolve_invite_slugs(client, slugs)
        found |= resolved

    return found

In [57]:
async def _check_text_url_entity(client, entity, message_text):
    """
    If it's a MessageEntityTextUrl, extract the URL, see if it has t.me pattern, 
    attempt to resolve.
    Returns a set of discovered usernames.
    """
    found = set()
    if entity.url:
        match = TMELINK_RE.search(entity.url)
        if match:
            slug = match.group(1).lstrip("+/")
            try:
                r = await client(ResolveUsernameRequest(slug))
                for ch in r.chats:
                    if getattr(ch, 'username', None):
                        found.add(ch.username)
            except:
                pass
    return found

In [58]:
def _check_mention_entity(entity, message_text):
    """
    If it's a mention or mention-name, parse out the substring from the message.
    Returns a set with the mention text (without '@').
    """
    found = set()
    from telethon.tl.types import MessageEntityMention, MessageEntityMentionName
    if isinstance(entity, (MessageEntityMention, MessageEntityMentionName)):
        mention_str = message_text[entity.offset : entity.offset + entity.length]
        mention_str = mention_str.lstrip('@')
        if mention_str:
            found.add(mention_str)
    return found

In [59]:

async def _parse_message_entities(client, ent, limit=100000):
    """
    Walk through message entities (text-url, @mention, etc.).
    Returns a set of discovered @usernames (without the leading '@').
    """
    found = set()

    async for m in _batched_iter_messages(ent, limit=limit):
        if not m.entities:
            continue

        txt = m.message or ""
        for e in m.entities:
            # 1) text-URL entities that contain a t.me link
            if isinstance(e, MessageEntityTextUrl):
                found |= await _check_text_url_entity(client, e, txt)
            # 2) explicit mention / mention-name entities
            else:
                found |= _check_mention_entity(e, txt)

    return found


In [60]:

async def _parse_button_links(client, ent, limit=100000):
    """
    Scan inline keyboard buttons for t.me links; resolve and return usernames.
    """
    found = set()

    async for m in _batched_iter_messages(ent, limit=limit):
        if not (m.reply_markup and getattr(m.reply_markup, "inline_keyboard", None)):
            continue

        for row in m.reply_markup.inline_keyboard:
            for button in row:
                if not button.url:
                    continue
                match = TMELINK_RE.search(button.url)
                if not match:
                    continue
                slug = match.group(1).lstrip("+/")
                try:
                    r = await client(ResolveUsernameRequest(slug))
                    for ch in r.chats:
                        if getattr(ch, "username", None):
                            found.add(ch.username)
                except Exception:
                    pass

    return found


In [61]:
### Cell 51 (code)

async def _participants_bounce(client, ent, scanned_user_ids, limit=200):
    """
    If 'ent' is a group-type entity (megagroup), fetch participants (up to 'limit').
    Then for each participant, scan their "about" text and their common chats 
    to discover potential public channels. We skip any user we've already scanned
    (via scanned_user_ids) to reduce duplicate calls.
    
    Returns a set() of discovered channel or group usernames.
    """
    found = set()

    # Check if it's a megagroup or group
    if hasattr(ent, 'megagroup') and ent.megagroup:
        from telethon.tl.functions.channels import GetParticipantsRequest
        from telethon.tl.types import ChannelParticipantsSearch

        try:
            offset = 0
            batch_size = 200  # Telethon's limit per request
            while True:
                result = await client(GetParticipantsRequest(
                    channel=ent,
                    filter=ChannelParticipantsSearch(''),
                    offset=offset,
                    limit=min(batch_size, limit),
                    hash=0
                ))
                users = result.users
                if not users:
                    break

                for u in users:
                    # Skip if we've scanned this user before
                    if u.id in scanned_user_ids:
                        continue
                    scanned_user_ids.add(u.id)

                    user_found = await _scan_user_for_channels(client, u)
                    found |= user_found

                offset += len(users)
                if len(users) < batch_size:
                    # No more participants to fetch or reached limit
                    break

                # If limit is very large, we might reduce it in steps
                limit -= len(users)
                if limit <= 0:
                    break

        except Exception as e:
            print("participants_bounce failed:", e)

    return found


In [62]:


import re
from telethon.tl.functions.users import GetFullUserRequest
from telethon.tl.types import Channel, Chat

async def _scan_user_for_channels(client, user_obj, pause_sec=0):
    """
    Given a Telethon 'User' object, fetch their FullUser data,
    then parse:
      1) The user's 'about' text for @usernames or t.me links
      2) The user's common chats for any channel/group that has a public username.
    
    Returns a set of discovered channel/group usernames (strings).
    """

    found = set()

    # Attempt to get full user info
    try:
        full_u = await client(GetFullUserRequest(user_obj))
    except:
        return found  # if we can't fetch the user, skip

    # (A) Parse 'about' for potential links
    about_text = (full_u.full_user.about or "").strip()
    USERNAME_RE = re.compile(r'@([\w\d_]{5,32})', re.IGNORECASE)
    TMELINK_RE  = re.compile(r'(?:https?://)?t\.me/([\w\d_+/]{5,64})', re.IGNORECASE)

    # 1. match @username
    for match in USERNAME_RE.findall(about_text):
        found.add(match)

    # 2. match t.me/...
    for match in TMELINK_RE.findall(about_text):
        found.add(match.lstrip("+/"))

    # (B) Parse common chats from 'full_u.chats'
    common_chats = getattr(full_u, 'chats', [])
    for c in common_chats:
        # c might be a Channel, Chat, or even a User
        if isinstance(c, (Channel, Chat)):
            uname = getattr(c, 'username', None)
            if uname:
                found.add(uname)

    # pause to avoid spamming the API
    if pause_sec:
        import asyncio
        await asyncio.sleep(pause_sec)

    return found


In [ ]:
main()

Please enter the code you received:  76978


Signed in successfully as Yipeng Wang; remember to not break the ToS or you will risk an account ban!
### ENTER discover_bfs ###
--- TOP OF LOOP --- @BigPumpsBinance
    progress → validated=   0 visited=  75 queue= 75
[DBG] @BigPumpsBinance: 2 raw neighbours (0 fwd, 1 mention, 1 textlink, 0 linked, 1 about/pinned, 0 invites, 0 entities, 0 buttons)
--- TOP OF LOOP --- @CoinCodeCap
    progress → validated=   1 visited=  76 queue= 75
[DBG] @CoinCodeCap: 15 raw neighbours (0 fwd, 4 mention, 2 textlink, 1 linked, 2 about/pinned, 0 invites, 9 entities, 0 buttons)
--- TOP OF LOOP --- @UniversalCryptoSignals
    progress → validated=   2 visited=  79 queue= 77
[DBG] @UniversalCryptoSignals: 16 raw neighbours (0 fwd, 9 mention, 6 textlink, 0 linked, 4 about/pinned, 1 invites, 6 entities, 0 buttons)
--- TOP OF LOOP --- @VerifiedCryptoNews
    progress → validated=   3 visited=  84 queue= 81
[DBG] @VerifiedCryptoNews: 18 raw neighbours (0 fwd, 2 mention, 2 textlink, 0 linked, 3 about/pinned, 0 

Server closed the connection: [Errno 60] Operation timed out
Graceful disconnection timed out, forcibly ignoring cleanup
Server closed the connection: [Errno 54] Connection reset by peer
Server closed the connection: [Errno 54] Connection reset by peer
Server resent the older message 7509554276996722689, ignoring
Server closed the connection: [Errno 54] Connection reset by peer
Server closed the connection: [Errno 54] Connection reset by peer
Server resent the older message 7509576103950749697, ignoring
Attempt 1 at connecting failed: OSError: [Errno 51] Network is unreachable
Attempt 2 at connecting failed: OSError: [Errno 51] Network is unreachable
Attempt 3 at connecting failed: OSError: [Errno 51] Network is unreachable
Attempt 4 at connecting failed: OSError: [Errno 51] Network is unreachable
Attempt 5 at connecting failed: OSError: [Errno 51] Network is unreachable
Attempt 6 at connecting failed: OSError: [Errno 51] Network is unreachable
Server closed the connection: [Errno 60] 

--- TOP OF LOOP --- @TheTruthAboutCancer_Vaccines
    progress → validated=  20 visited= 176 queue=145
[DBG] @TheTruthAboutCancer_Vaccines: 32 raw neighbours (0 fwd, 14 mention, 11 textlink, 0 linked, 1 about/pinned, 7 invites, 13 entities, 0 buttons)
--- TOP OF LOOP --- @DrJaneRuby
    progress → validated=  21 visited= 182 queue=150
[DBG] @DrJaneRuby: 115 raw neighbours (0 fwd, 55 mention, 51 textlink, 1 linked, 2 about/pinned, 19 invites, 53 entities, 0 buttons)
--- TOP OF LOOP --- @PeterMcCullough
    progress → validated=  22 visited= 204 queue=171
[DBG] @PeterMcCullough: 11 raw neighbours (0 fwd, 5 mention, 4 textlink, 0 linked, 1 about/pinned, 3 invites, 4 entities, 0 buttons)
--- TOP OF LOOP --- @FLCCC_Alliance
    progress → validated=  23 visited= 205 queue=171
    ✖ REJECT FLCCC_Alliance en_ratio=0.01 < 0.2
--- TOP OF LOOP --- @VigilantFox
    progress → validated=  23 visited= 205 queue=170
[DBG] @VigilantFox: 79 raw neighbours (0 fwd, 37 mention, 34 textlink, 1 linked, 2 a

Server closed the connection: [Errno 54] Connection reset by peer
Server resent the older message 7509996555889563649, ignoring
Server closed the connection: [Errno 54] Connection reset by peer
Server closed the connection: [Errno 54] Connection reset by peer
Server closed the connection: [Errno 54] Connection reset by peer
Server closed the connection: [Errno 54] Connection reset by peer
